In [1]:
import re

import polars as pl
import wordcloud
from frontend.configs import CATEGORY_ID_MAP_PATH, VIDEO_DETAILS_JSON_PATH
from frontend.youtube import IngestYtHistory
from matplotlib import pyplot as plt
from plotly import express as px

In [2]:
pl.Config.set_fmt_str_lengths(40)

polars.config.Config

In [ ]:
df = IngestYtHistory.from_ingested_data()
df.head()

In [4]:
vdf = pl.read_json(VIDEO_DETAILS_JSON_PATH)
vdf.head()

categoryId,channelId,channelTitle,description,duration,id,publishedAt,tags,title
str,str,str,str,str,str,str,list[str],str
"""28""","""UCs5Y5_7XK8HLDX0SLNwkd3w""","""Visual Studio Code""","""#shorts""","""PT38S""","""4dgSwnKtVLo""","""2023-09-22T05:12:53""","[""vscode"", ""visual studio code"", ""vs code""]","""Arrow Function: Think Backwards"""
"""28""","""UC46xhU1EH7aywEgvA9syS3w""","""anthonywritescode""","""it's here! with ugly new typing syntax…","""PT41M42S""","""IV8OZY4194U""","""2023-08-02T15:00:25""",null,"""python 3.12 release highlights (beginne…"
"""27""","""UCsDTy8jvHcwMvSZf_JGi-FA""","""Abhi and Niyu""","""To support our work you can consider bu…","""PT14M39S""","""4uM7RIfMLK4""","""2023-10-07T13:00:12""","[""abhi and niyu"", ""abhi and niyu latest"", … ""abi n niu""]","""Australia's economy is WEIRD | Australi…"
"""28""","""UCQV-7R4qhAJXxdkrWNOglDQ""","""Dekho Isko""","""Join me on Social media:- 📷 Instagram •…","""PT4M41S""","""uWLooUi8in0""","""2023-09-06T16:52:42""","[""india rename"", ""india renamed as bharat"", … ""rename india as bharat""]","""Will India Change its Name to Bharat Pe…"
"""24""","""UCgbzclo4Mfy_D68w_Bm_xHg""","""Tried&Refused Productions.""","""There is always so much to watch but I …","""PT10M33S""","""diajsqMk-mo""","""2023-09-02T06:45:02""","[""maaveeran"", ""movie"", … ""yogi babu""]","""3 Amazing Indian Movies In 2023 That De…"


In [5]:
cat_id_df = pl.read_json(CATEGORY_ID_MAP_PATH).transpose(
    include_header=True,
    header_name="categoryId",
    column_names=["categoryName"],
)
cat_id_df.head()

categoryId,categoryName
str,str
"""01""","""Film & Animation"""
"""02""","""Autos & Vehicles"""
"""10""","""Music"""
"""15""","""Pets & Animals"""
"""17""","""Sports"""


In [6]:
mdf = df.join(
    vdf.drop("title", "channelTitle", "channelId"), left_on="videoId", right_on="id"
).join(cat_id_df, on="categoryId")
mdf.head()

title,titleUrl,time,fromYtSearchHistActivity,fromYtWatchHistActivity,fromWebAppActivity,videoId,channelTitle,channelId,year,month,weekday,hour,isShorts,titleTags,titleEmojis,contentType,categoryId,description,duration,publishedAt,tags,categoryName
str,str,"datetime[μs, UTC]",bool,bool,bool,str,str,str,i64,i64,i64,i64,bool,list[str],list[str],str,str,str,str,str,list[str],str
"""The EU Will Break Apple.""","""https://youtu.be/Hod69Y1aVlY""",2023-11-14 03:33:58.802 UTC,false,true,false,"""Hod69Y1aVlY""","""TechLinked""","""UCeeFfhMcJa1kjtfZAGskOCA""",2023,11,2,3,false,[],[],"""News""","""28""","""See why Gcore is an industry leader in …","""PT7M55S""","""2023-11-14T02:33:35""","[""Linus"", ""LinusTechTips"", … ""Google Play Store""]","""Science & Technology"""
"""Why are Rajasthanis unemployed?""","""https://youtu.be/_GkAzfDD-uc""",2023-11-14 03:19:08.026 UTC,false,true,false,"""_GkAzfDD-uc""","""Soch by Mohak Mangal""","""UCz4a7agVFr1TxU-mpAP8hkw""",2023,11,2,3,false,[],[],"""News""","""27""","""Follow me on LinkedIn: https://www.link…","""PT23M1S""","""2023-11-13T09:30:08""","[""India""]","""Education"""
"""Google is paying Samsung - Good news fo…","""https://youtu.be/3K9miZLIBfg""",2023-11-14 03:11:42.709 UTC,false,true,false,"""3K9miZLIBfg""","""Tech Junkies""","""UCn4fhtzOgtoEKIrAAV4yCMA""",2023,11,2,3,false,[],[],"""News""","""28""","""Google is bribing Samsung for all andro…","""PT10M30S""","""2023-11-13T12:46:56""","[""tech junkies"", ""google"", … ""elon musk""]","""Science & Technology"""
"""Apple Ultra Saved Diabetic, Qualcomm Sa…","""https://youtu.be/Z5F2Q1h2OY4""",2023-11-13 15:34:50.137 UTC,false,true,false,"""Z5F2Q1h2OY4""","""Dekho Isko""","""UCQV-7R4qhAJXxdkrWNOglDQ""",2023,11,1,15,false,[],[],"""News""","""28""","""Join me on Social media:- 📷 Instagram •…","""PT7M4S""","""2023-11-13T15:30:11""","[""Apple ultra watch"", ""apple watch life save"", … ""daily internet news""]","""Science & Technology"""
"""Memer Job 1 Lakh Salary, Cybertruck No …","""https://youtu.be/CP5bWH51VNM""",2023-11-13 02:38:33.505 UTC,false,true,false,"""CP5bWH51VNM""","""Dekho Isko""","""UCQV-7R4qhAJXxdkrWNOglDQ""",2023,11,1,2,false,[],[],"""News""","""28""","""Join me on Social media:- 📷 Instagram •…","""PT7M2S""","""2023-11-12T15:40:17""","[""cybertruck"", ""tesla cybertruck"", … ""dekho isko""]","""Science & Technology"""


In [ ]:
def handle_duration(x: str):
    total_sec = 0
    # Hours
    hr = re.search(r"(\d+)H", x)
    total_sec += int(hr.group(1)) * 3600 if hr else 0
    # Minutes
    min = re.search(r"(\d+)M", x)  # noqa: A001
    total_sec += int(min.group(1)) * 60 if min else 0
    # Seconds
    sec = re.search(r"(\d+)S", x)
    total_sec += int(sec.group(1)) if sec else 0
    return total_sec

In [8]:
mdf = mdf.with_columns(
    pl.col("publishedAt").str.to_datetime(),
    pl.col("duration").map_elements(handle_duration, pl.Int64).alias("durationInSec"),
    pl.col("duration").str.contains("D").alias("durationHasDay"),
    pl.col("tags").is_null().add(pl.col("isShorts")).cast(bool).alias("isShorts"),
    pl.col("hour")
    .cut(
        breaks=[5, 12, 17, 21],
        labels=["Night", "Morning", "Afternoon", "Evening", "Night"],
        left_closed=True,
    )
    .alias("daytime"),
).drop("duration", "categoryId")
mdf.head()

title,titleUrl,time,fromYtSearchHistActivity,fromYtWatchHistActivity,fromWebAppActivity,videoId,channelTitle,channelId,year,month,weekday,hour,isShorts,titleTags,titleEmojis,contentType,description,publishedAt,tags,categoryName,durationInSec,durationHasDay,daytime
str,str,"datetime[μs, UTC]",bool,bool,bool,str,str,str,i64,i64,i64,i64,bool,list[str],list[str],str,str,datetime[μs],list[str],str,i64,bool,cat
"""The EU Will Break Apple.""","""https://youtu.be/Hod69Y1aVlY""",2023-11-14 03:33:58.802 UTC,false,true,false,"""Hod69Y1aVlY""","""TechLinked""","""UCeeFfhMcJa1kjtfZAGskOCA""",2023,11,2,3,false,[],[],"""News""","""See why Gcore is an industry leader in …",2023-11-14 02:33:35,"[""Linus"", ""LinusTechTips"", … ""Google Play Store""]","""Science & Technology""",475,false,"""Night"""
"""Why are Rajasthanis unemployed?""","""https://youtu.be/_GkAzfDD-uc""",2023-11-14 03:19:08.026 UTC,false,true,false,"""_GkAzfDD-uc""","""Soch by Mohak Mangal""","""UCz4a7agVFr1TxU-mpAP8hkw""",2023,11,2,3,false,[],[],"""News""","""Follow me on LinkedIn: https://www.link…",2023-11-13 09:30:08,"[""India""]","""Education""",1381,false,"""Night"""
"""Google is paying Samsung - Good news fo…","""https://youtu.be/3K9miZLIBfg""",2023-11-14 03:11:42.709 UTC,false,true,false,"""3K9miZLIBfg""","""Tech Junkies""","""UCn4fhtzOgtoEKIrAAV4yCMA""",2023,11,2,3,false,[],[],"""News""","""Google is bribing Samsung for all andro…",2023-11-13 12:46:56,"[""tech junkies"", ""google"", … ""elon musk""]","""Science & Technology""",630,false,"""Night"""
"""Apple Ultra Saved Diabetic, Qualcomm Sa…","""https://youtu.be/Z5F2Q1h2OY4""",2023-11-13 15:34:50.137 UTC,false,true,false,"""Z5F2Q1h2OY4""","""Dekho Isko""","""UCQV-7R4qhAJXxdkrWNOglDQ""",2023,11,1,15,false,[],[],"""News""","""Join me on Social media:- 📷 Instagram •…",2023-11-13 15:30:11,"[""Apple ultra watch"", ""apple watch life save"", … ""daily internet news""]","""Science & Technology""",424,false,"""Afternoon"""
"""Memer Job 1 Lakh Salary, Cybertruck No …","""https://youtu.be/CP5bWH51VNM""",2023-11-13 02:38:33.505 UTC,false,true,false,"""CP5bWH51VNM""","""Dekho Isko""","""UCQV-7R4qhAJXxdkrWNOglDQ""",2023,11,1,2,false,[],[],"""News""","""Join me on Social media:- 📷 Instagram •…",2023-11-12 15:40:17,"[""cybertruck"", ""tesla cybertruck"", … ""dekho isko""]","""Science & Technology""",422,false,"""Night"""


### Sunbrust between `caetgoryName` and `channelTitle`


In [ ]:
px.sunburst(
    mdf.group_by(["channelTitle", "categoryName"]).count(),
    path=["categoryName", "channelTitle"],
    values="count",
    title="Video Distribution by Category and Channel",
)

### WorldCloud of `titleEmojis`


In [9]:
emoji_freq = dict(
    zip(
        *mdf["titleEmojis"]
        .explode()
        .value_counts(sort=True)
        .drop_nulls()
        .to_dict(as_series=False)
        .values(),
        strict=False,
    )
)
print(emoji_freq)

{'😱': 11, '👏': 9, '😮': 8, '🇮🇳': 6, '😂': 3, '🤯': 3, '🔥': 3, '⭐': 2, '🙏': 2, '🍀': 2, '☹️': 2, '😰': 2, '👀': 1, '❤️': 1, '😎': 1, '📢': 1, '🔴': 1, '⚡': 1, '😅': 1, '🛑': 1, '⚠️': 1, '💡': 1, '🎁': 1, '🇲🇾': 1, '🇲🇽': 1, '🤨': 1, '😭': 1, '💼': 1, '⚖️': 1, '🤔': 1}


In [ ]:
cloud = wordcloud.WordCloud("Menlo", height=800, width=800).generate_from_frequencies(
    emoji_freq
)
plt.imshow(cloud, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud of Emojis in Titles")
plt.show()

### Wordcloud of `titleTags`


In [19]:
tags_text = " ".join(
    mdf["titleTags"]
    .explode()
    .drop_nulls()
    .str.strip_prefix("#")
    .str.to_lowercase()
    .to_list()
)
tags_text

'10 07 06 06 05 python code windows development python lambda vscode code javascript programming code vscode vscode 04 semiconductor business startup 03 6 5 4 3 2 1 startup business entrepreneurship iphone iphone15 bigbilliondays2023 gif bbd 357 043 iphone iphone15 iphone15promax startup business entrepreneurship 01 3 modi 434 102 427 423 433 550 430 368 164 534 540 548 166 167'

In [ ]:
cloud = wordcloud.WordCloud(height=800, width=800).generate(tags_text)
plt.figure(figsize=(10, 10))
plt.imshow(cloud, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud of Emojis in Titles")
plt.show()

### User's watching patterns


In [ ]:
px.sunburst(
    mdf.group_by(["daytime", "categoryName"]).count(),
    path=["daytime", "categoryName"],
    values="count",
    title="User's watching behaviour during daytime",
)

#### Patterns in `month` and `year`


In [ ]:
__year = 2023

px.sunburst(
    mdf.filter(pl.col("year") == __year).group_by(["month", "categoryName"]).count(),
    path=["month", "categoryName"],
    values="count",
    title="Watching patterns for months.",
)

### EDA of video `duration`


In [ ]:
px.pie(
    mdf["isShorts"]
    .map_elements(lambda x: "Shorts Video" if x else "Long Videos")
    .value_counts(),
    names="isShorts",
    values="counts",
    title="Ratio between Shorts and Long Form Video",
)

Suggestion to get insights from `duration` column:

- **Average duration by content type:** This can help you identify the types of content that the user typically watches
  for longer periods of time. For example, you might find that the user watches longer videos on entertainment channels
  than on news channels.

- **Average duration by time period:** This can help you identify when the user is most likely to watch longer videos.
  For example, you might find that the user watches longer videos in the evening than in the morning.

- **Distribution of durations:** This can help you understand how the user's watch time is distributed across different
  video lengths. For example, you might find that the user watches a lot of short videos, but also watches a few long
  videos each week.

- **Correlation between duration and other features:** This can help you identify other factors that are related to how
  long the user watches videos. For example, you might find that the user watches longer videos on channels that they
  subscribe to, or that they watch longer videos when they are using the YouTube app on their phone.


In [ ]:
px.sunburst(
    mdf.group_by("categoryName", "channelTitle").agg(
        pl.col("durationInSec").mean().cast(int).alias("durationMean"),
    ),
    path=["categoryName", "channelTitle"],
    values="durationMean",
    title="Average Video Duration by Category and Channel",
)

In [ ]:
px.sunburst(
    mdf.group_by("categoryName", "channelTitle").agg(
        pl.col("isShorts").sum(),
    ),
    path=["categoryName", "channelTitle"],
    values="isShorts",
    title="Distribution of Shorts by Channels",
)